# Attempt 6
### Use kerastuner for best hyperparameters
### Activation Function: tanh
### 100 epochs, Two hidden layers with 7 and 9 units (neurons) respectively.

## Preprocessing

In [103]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
import keras_tuner as kt
#from kerastuner.tuners import RandomSearch

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

# Clear session to reset model name counter
K.clear_session

<function keras.backend.clear_session()>

In [104]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], inplace=True, axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [105]:
# Determine the number of unique values in each column.
application_cat = application_df.dtypes.index.tolist()
application_df[application_cat].nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [106]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [107]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff_value = 528

# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts < cutoff_value].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [108]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts.head(20)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: CLASSIFICATION, dtype: int64

In [109]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = classification_counts[classification_counts > 1]
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [110]:
# Choose a cutoff value and create a list of classifications to be replaced
cutoff_value = 1883

# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < cutoff_value].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
C1370        1
C1236        1
C2190        1
C4200        1
C5200        1
C1248        1
C2600        1
C1580        1
C6100        1
C1820        1
C1900        1
C2570        1
C1570        1
C3700        1
C2500        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: CLASSIFICATION, dtype: int64

In [111]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df[["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "STATUS", "INCOME_AMT", "SPECIAL_CONSIDERATIONS", "ASK_AMT"]])
dummies.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [112]:
# Split our preprocessed data into our features and target arrays
X = dummies.values
y = application_df["IS_SUCCESSFUL"].values


In [113]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [82]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Use kerastuner

In [114]:
# Create method for hyperparameter options

def create_model(hp):
    nn_model = keras.Sequential()

    # Allow kerastuner to decide on the activation function for hidden layers
    activation = hp.Choice("activation",["relu", "tanh", "sigmoid"])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int("first_units", 
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=69))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int("num_of_layers", 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int("units_" + str(i),
            min_value=1,
            max_value=10,
            step=2), activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    return nn_model

In [115]:
# Set parameters

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    directory="KerasTuner",
    project_name="NN_Hyperband"
)

In [116]:
# Run kerastuner for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 60 Complete [00h 00m 59s]
val_accuracy: 0.7328279614448547

Best val_accuracy So Far: 0.7328279614448547
Total elapsed time: 00h 24m 37s
INFO:tensorflow:Oracle triggered exit


In [117]:
# Get best model parameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 5,
 'num_of_layers': 3,
 'units_0': 9,
 'units_1': 1,
 'units_2': 5,
 'units_3': 5,
 'units_4': 9,
 'units_5': 3,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

## Compile, Train and Evaluate the Model

In [118]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=9, activation="tanh", input_dim=len(X_train_scaled[0])))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 9)                 630       
                                                                 
 dense_6 (Dense)             (None, 1)                 10        
                                                                 
 dense_7 (Dense)             (None, 5)                 10        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 656
Trainable params: 656
Non-trainable params: 0
_________________________________________________________________


In [119]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [120]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5960 - accuracy: 0.7051
Epoch 2/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5741 - accuracy: 0.7282
Epoch 3/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5704 - accuracy: 0.7288
Epoch 4/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5666 - accuracy: 0.7282
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5638 - accuracy: 0.7293
Epoch 6/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5608 - accuracy: 0.7287
Epoch 7/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5580 - accuracy: 0.7294
Epoch 8/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5562 - accuracy: 0.7295
Epoch 9/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5550 - accuracy: 0.7301
Epoch 10/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5540 - accuracy: 0.7311

In [121]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5578 - accuracy: 0.7261 - 460ms/epoch - 2ms/step
Loss: 0.5578396320343018, Accuracy: 0.726064145565033


In [102]:
# Export our model to HDF5 file
nn.save("Resources/AlphabetSoupCharity_Optimization_6.h5")